<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/sparsity_selectivity_4_HL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

root_dir = './'
torchvision.datasets.MNIST(root=root_dir,download=True)

--2021-03-16 01:00:51--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-03-16 01:00:51--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [            <=>     ]  33.20M  6.42MB/s    in 15s     

2021-03-16 01:01:06 (2.28 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./
    Split: Train

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
torch.manual_seed(1234)
np.random.seed(1234)

In [5]:
mnist_trainset = torchvision.datasets.MNIST(root=root_dir, train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = torchvision.datasets.MNIST(root=root_dir, 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

In [6]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 256)
        self.linear_3 = torch.nn.Linear(256, 256)
        self.linear_4 = torch.nn.Linear(256, 256)
        self.linear_5 = torch.nn.Linear(256, 10)
        self.sigmoid12  = torch.nn.Sigmoid()
        self.sigmoid23  = torch.nn.Sigmoid()
        self.sigmoid34  = torch.nn.Sigmoid()
        self.sigmoid45  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        x = self.linear_2(x)
        x = self.sigmoid23(x)
        x = self.linear_3(x)
        x = self.sigmoid34(x)
        x = self.linear_4(x)
        x = self.sigmoid45(x)
        pred = self.linear_5(x)
        return pred
# ************* modify this section for later use *************

In [7]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [8]:
def sparsity_calculator(final_spareness):
    sparseness_list = list()
    for single_epoch_spareness in final_spareness:

        hidden_layer_activation_list = single_epoch_spareness
        hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
        layer_activations_list = torch.reshape(hidden_layer_activation_list, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)

    return sparseness_list

In [9]:
def selectivity(hidden_layer_each_neuron):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(256)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(256)]

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)

    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [10]:
# ************* modify this section for later use *************
# add a parameter to the function and calculate avg and std. Do not forget to change division by 2, 3, 4, or 5 
def avg_std_calculator(_hidden_layer_each_neuron_12, _hidden_layer_each_neuron_23, _hidden_layer_each_neuron_34, _hidden_layer_each_neuron_45):

    avg_selectivity12, std_selectivity12 = selectivity(_hidden_layer_each_neuron_12)
    avg_selectivity23, std_selectivity23 = selectivity(_hidden_layer_each_neuron_23)
    avg_selectivity34, std_selectivity34 = selectivity(_hidden_layer_each_neuron_34)
    avg_selectivity45, std_selectivity45 = selectivity(_hidden_layer_each_neuron_45)

    final_selectivity_avg = (avg_selectivity12 + avg_selectivity23 + avg_selectivity34 + avg_selectivity45) / 4
    final_selecvitity_std = (std_selectivity12 + std_selectivity23 + std_selectivity34 + std_selectivity45) / 4

    return final_selectivity_avg, final_selecvitity_std
# ************* modify this section for later use *************

In [11]:
def model_factory(optimizer_name):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model()
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    # chagen sigmoid34 an 's34'
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    my_model.sigmoid23.register_forward_hook(get_activation(my_model, 's23'))
    my_model.sigmoid34.register_forward_hook(get_activation(my_model, 's34'))
    my_model.sigmoid45.register_forward_hook(get_activation(my_model, 's45'))
    # ************* modify this section for later use *************

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, sparsity_avg, selectivity_list_avg, selectivity_list_std = selectivity_trainer(optimizer=my_optimizer, model=my_model)
    # ************* modify this section for later use *************
    # change name of the file 
    file_saver = open(f"4HL_selectivity_sparsity_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(sparsity_avg)+'\n'+str(selectivity_list_avg)+'\n'+str(selectivity_list_std)+'\n\n')
    file_saver.close()

    # ************* modify this section for later use *************
    if optimizer_name == 'Adadelta':
        !cp 4HL_selectivity_sparsity_Adadelta.txt /content/drive/MyDrive
    
    elif optimizer_name == 'Adagrad':
        !cp 4HL_selectivity_sparsity_Adagrad.txt /content/drive/MyDrive

    elif optimizer_name == 'SGD':
        !cp 4HL_selectivity_sparsity_SGD.txt /content/drive/MyDrive

    elif optimizer_name == 'Adam':
        !cp 4HL_selectivity_sparsity_Adam.txt /content/drive/MyDrive
    # ************* modify this section for later use *************

    else:
        print("ERROR")

In [12]:
no_epochs = 50
def selectivity_trainer(optimizer, model):

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    selectivity_avg_list = list()
    selectivity_std_list = list()

    # ************* modify this section for later use *************
    final_spareness_12 = list()
    final_spareness_23 = list()
    final_spareness_34 = list()
    final_spareness_45 = list()
    # ************* modify this section for later use *************

    for epoch in range(no_epochs):

        # ************* modify this section for later use *************
        hidden_layer_activation_list_12 = list()
        hidden_layer_activation_list_23 = list()
        hidden_layer_activation_list_34 = list()
        hidden_layer_activation_list_45 = list()
        # ************* modify this section for later use *************

        # ************* modify this section for later use *************
        hidden_layer_each_neuron_12 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_12 = np.array(hidden_layer_each_neuron_12)

        hidden_layer_each_neuron_23 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_23 = np.array(hidden_layer_each_neuron_23)

        hidden_layer_each_neuron_34 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_34 = np.array(hidden_layer_each_neuron_34)

        hidden_layer_each_neuron_45 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_45 = np.array(hidden_layer_each_neuron_45)
        # ************* modify this section for later use *************


        total_train_loss = 0
        total_test_loss = 0

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # ***************** sparsity calculation ***************** #
            hidden_layer_activation_list_12.append(model.layer_activations['s12'])
            hidden_layer_activation_list_23.append(model.layer_activations['s23'])
            hidden_layer_activation_list_34.append(model.layer_activations['s34'])
            hidden_layer_activation_list_45.append(model.layer_activations['s45'])

            # ************* modify this section for later use *************
            # Do not forget to change hidden_layer_each_neuron_12 name 
            for activation, label in zip(model.layer_activations['s12'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_12[i][label].append(activation[i])

            for activation, label in zip(model.layer_activations['s23'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_23[i][label].append(activation[i])
            
            for activation, label in zip(model.layer_activations['s34'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_34[i][label].append(activation[i])

            for activation, label in zip(model.layer_activations['s45'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_45[i][label].append(activation[i])

        # Add one more parameter 
        selectivity_avg, selecvitity_std = avg_std_calculator(hidden_layer_each_neuron_12, hidden_layer_each_neuron_23, hidden_layer_each_neuron_34, hidden_layer_each_neuron_45)
        # ************* modify this section for later use *************
        
        selectivity_avg_list.append(selectivity_avg)
        selectivity_std_list.append(selecvitity_std)

        final_spareness_12.append(hidden_layer_activation_list_12)
        final_spareness_23.append(hidden_layer_activation_list_23)
        final_spareness_34.append(hidden_layer_activation_list_34)
        final_spareness_45.append(hidden_layer_activation_list_45)
        # ***************** sparsity calculation ***************** #

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

    # ***************** sparsity calculation ***************** #
    sparsity_list12 = sparsity_calculator(final_spareness_12)
    sparsity_list23 = sparsity_calculator(final_spareness_23)
    sparsity_list34 = sparsity_calculator(final_spareness_34)
    sparsity_list45 = sparsity_calculator(final_spareness_45)

    print(sparsity_list12)
    print(sparsity_list23)
    print(sparsity_list34)
    print(sparsity_list45)

    average_sparsity = list()
    for i in range(no_epochs):
        average_sparsity.append( (sparsity_list12[i].item() + sparsity_list23[i].item() + sparsity_list34[i].item() + sparsity_list45[i].item()) / 4 )

    # ***************** sparsity calculation ***************** #

    print("average_sparsity:", average_sparsity)

    return test_acc, average_sparsity, selectivity_avg_list, selectivity_std_list

# Adadelta

In [13]:
model_factory('Adadelta')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=256, bias=True)
  (linear_4): Linear(in_features=256, out_features=256, bias=True)
  (linear_5): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
  (sigmoid34): Sigmoid()
  (sigmoid45): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 2.30746806, Test Loss: 2.30202827, Test Accuracy: 0.09580000

Epoch: 2/50, Train Loss: 2.30127321, Test Loss: 2.29988618, Test Accuracy: 0.11350000

Epoch: 3/50, Train Loss: 2.18322148, Test Loss: 1.78346998, Test Accuracy: 0.35970000

Epoch: 4/50, Train Loss: 1.38807317, Test Loss: 1.05196509, Test Accuracy: 0.58940000

Epoch: 5/50, Train Loss: 0.73925288, Test Loss: 0.40720915, Test Accuracy: 0.89020000

Epoch:

# AdaGrad

In [14]:
model_factory('Adagrad')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=256, bias=True)
  (linear_4): Linear(in_features=256, out_features=256, bias=True)
  (linear_5): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
  (sigmoid34): Sigmoid()
  (sigmoid45): Sigmoid()
)
my_optimizer: Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0
    lr: 0.1
    lr_decay: 0
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 2.41819022, Test Loss: 2.32539536, Test Accuracy: 0.09800000

Epoch: 2/50, Train Loss: 2.32925108, Test Loss: 2.33123923, Test Accuracy: 0.09740000

Epoch: 3/50, Train Loss: 2.32166392, Test Loss: 2.32314516, Test Accuracy: 0.11350000

Epoch: 4/50, Train Loss: 2.31882394, Test Loss: 2.32085734, Test Accuracy: 0.09800000

Epoch: 5/50, Train Loss: 2.31751979, Test Loss: 2.32146750

# SGD


In [15]:
model_factory('SGD')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=256, bias=True)
  (linear_4): Linear(in_features=256, out_features=256, bias=True)
  (linear_5): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
  (sigmoid34): Sigmoid()
  (sigmoid45): Sigmoid()
)
my_optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 2.32315335, Test Loss: 2.30903590, Test Accuracy: 0.09820000

Epoch: 2/50, Train Loss: 2.30906891, Test Loss: 2.30911063, Test Accuracy: 0.10320000

Epoch: 3/50, Train Loss: 2.30566493, Test Loss: 2.30473551, Test Accuracy: 0.11350000

Epoch: 4/50, Train Loss: 2.30376772, Test Loss: 2.30548497, Test Accuracy: 0.10100000

Epoch: 5/50, Train Loss: 2.30315410, Test Loss: 2.30266628, Test Accuracy

# Adam

In [16]:
model_factory('Adam')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=256, bias=True)
  (linear_4): Linear(in_features=256, out_features=256, bias=True)
  (linear_5): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
  (sigmoid34): Sigmoid()
  (sigmoid45): Sigmoid()
)
my_optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 0.63060462, Test Loss: 0.26807360, Test Accuracy: 0.92430000

Epoch: 2/50, Train Loss: 0.20602424, Test Loss: 0.17091358, Test Accuracy: 0.95030000

Epoch: 3/50, Train Loss: 0.13963423, Test Loss: 0.13862532, Test Accuracy: 0.96050000

Epoch: 4/50, Train Loss: 0.10679276, Test Loss: 0.11708062, Test Accuracy: 0.96550000

Epoch: 5/50, Train Loss: 0.08605078, Test Loss: 0.10161783, Test 